# Silicium-Aluminum soil concentration
This is a notebook file that discusses the Si-Al soil concentration model. Functions are defined in a seperate file: *SiAl_functions.ipynb*.
$$
    \DeclareMathOperator{Si}{Si}
    \DeclareMathOperator{Al}{Al}
    \DeclareMathOperator{exp}{e}
    \DeclareMathOperator{AlkExSi}{AlkExSi}
    \DeclareMathOperator{SiAl}{Si/Al}
$$

In [ ]:
%run ./SiAl_functions.ipynb

# Model
We have measurements of of the silicium ($\Si$) and aluminum ($\Al$) concentration in a soil sample at different time steps. We now wish to model both concentrations as a function of time using the following expansion:
\begin{align*}
    \Si(t) &= \sum_{i = 1}^n\AlkExSi_i(1 - \exp^{-k_i(t - t_0)}) + b(t - t_0)\\
    \Al(t) &= \sum_{i = 1}^n\frac{\AlkExSi_i}{\SiAl_i}(1 - \exp^{-k_i(t - t_0)}) + \frac{b}{\SiAl_0}(t - t_0)\\
\end{align*}
Both formulas consist of a linear term plus a number of exponentially decaying parts. The number of the exponentially decaying parts can be chosen in the model. The number of parameters in the model equals $3n + 3$, namely:
\begin{align*}
    k_i & \text{ for }i\in\{1, \ldots, n\}\\
    \AlkExSi_i & \text{ for }i\in\{1, \ldots, n\}\\
    \SiAl_i & \text{ for }i\in\{1, \ldots, n\}\\
    b &\\
    \SiAl_0 &\\
    t_0 &
\end{align*}

# Cost function

Let $\overline{\Si}$ and $\overline{\Al}$ be the vectors in $\mathbb{R}^m$ containing the observation data at the measurement times $\{t_1, \ldots, t_m\}$ and $\Si$ and $\Al$ the vectors in $\mathbb{R}^m$ with our model predictions (for a certain choice of parameters).
The goal now is to minimize the cost function
$$
    J = \left\|\Si - \overline{\Si}\right\|_2^2 + \left\|\Al - \overline{\Al}\right\|_2^2
$$
with respect to all the parameters.

# Read the data

We assume the data is given in a .csv file with three columns: time, Si measurements and Al meaurements.

The first row may contain names for the columns, but for consitency, these are replaced with the names 'time', 'SiBar' and 'AlBar'. If no columns names are present, the option 'header = None' should be used in stead of 'header = 0'.

We assume that decimals are denoted with a '.' and that the delimiter between the columns is a ';'. This can be changed by changing the 'delimiter = ';'' argument.

### Select the data file:
Either give a relative path or a full path to the data file.

In [ ]:
# Relative or full path to the data file:
filename = 'example_data.csv'     

In [ ]:
# Read data with pandas: (possibly change the delimiter and header input)
data = pd.read_csv(filename, delimiter = ';', header = 0, names = ['time', 'SiBar', 'AlBar'])  
data = data.dropna(how = 'any')

# Cast data to ndarray for later calculations with numpy:
data_np = data.as_matrix()
time = data_np[:, 0]
SiBar = data_np[:, 1]
AlBar = data_np[:, 2]

#
nmeas = time.size
nsamples = 2*time.size

# Show a figure of the measured data:
plt.figure()
lineSiBar = plt.plot(time, SiBar, 'r,', label = '$\overline{Si}$')
lineAlBar = plt.plot(time, AlBar, 'b,', label = '$\overline{Al}$')
plt.title('Measurement data')
plt.xlabel('Time')
plt.ylabel('Concentration ($\mu$mol/L)')
plt.legend(loc = 4)
plt.show()

# Functions

The functions needed for the optimization routine and the statistical analysis of the data are defined in the file "SiAlProblem_Functions.ipyng". We wil assume that the $3n + 3$ parameters are given as a ndarray of shape (3n + 3, ) in the following order:

$$
    \left[k_1, \ldots, k_n, \AlkExSi_1, \ldots, \AlkExSi_n, \SiAl_1, \ldots, \SiAl_n, b, \SiAl_0, t_0\right]
$$

# Initial estimate

Because for example any permuation of the parameter tupples $(\alpha_i, \beta_i, k_i)$ is an equivalent reformulation of the optimization problem, it is to be expected that the cost function $J$ will have multiple local minima and maxima. We therefore need a good initial guess of the optimal set of parameters.

Consider for now the case $n = 1$, where
$$
    \Si(t) = \AlkExSi(1 - \exp^{-kt}) + bt.
$$
The linear part of this formula is given by

$$
    \AlkExSi + bt
$$

which can be estimated by a simple linear regression model on the second part of the data (where t is big). The exponential decaying part can then by approximated by looking at
$$
    \exp^{-kt} = (\AlkExSi + bt) - \Si(t).
$$
and thus by performing an exponential regression on the data $(t_i, \AlkExSi + bt - \overline{Si}(t_i))$. By doing the same for the $\Al$ concentration we can also get estimates for $\SiAl$ and $\SiAl_0$.

When $1 < n$ we base our initial estimate on the parameters found for the model $n - 1$.

When $1 < n$ we propose to take all the parameters equal to their $n = 1$ counterpart, except for $\AlkExSi$, which we replace with $\AlkExSi/n$. This way the inital estimate is independent of $n$.

In [ ]:
# Select the linear part of the data:
M = len(time[max(time) - 10 < time])                                                    # Take the last 10 minutes of measurents.

N = SiBar[M:].size

# Linear regression on SiBar data:
slopeSi = (np.sum(time[M:]*SiBar[M:]) - np.sum(time[M:])*np.sum(SiBar[M:])/N)/(np.sum(time[M:]**2) - np.sum(time[M:])**2/N)
intSi = np.sum(SiBar[M:])/N - slopeSi*np.sum(time[M:])/N
SiLin = intSi + slopeSi*time

# Linear regression on AlBar data:
slopeAl = (np.sum(time[M:]*AlBar[M:]) - np.sum(time[M:])*np.sum(AlBar[M:])/N)/(np.sum(time[M:]**2) - np.sum(time[M:])**2/N)
intAl = np.sum(AlBar[M:])/N - slopeAl*np.sum(time[M:])/N
AlLin = intAl + slopeAl*time

# Select the exponential part of the data:
M = len(time[time < 10])                                                            # Take the first 10 minutes of measurements. 
N = SiBar[M:].size

# Exponential decay on SiBar data:
index1 = np.where(SiLin - SiBar > 0)      # We can't take the log of negative values. However, due to the noise this might occur.
index2 = np.where(index1[0] < M)
index3 = index1[0][index2[0]]
k0Si = -np.sum((np.log((SiLin - SiBar)[index3]) - np.log(intSi))*time[index3])/np.sum(time[index3]**2)

# Exponential decay on AlBar data:
index1 = np.where(AlLin - AlBar > 0)      # We can't take the log of negative values. However, due to the noise this might occur.
index2 = np.where(index1[0] < M)
index3 = index1[0][index2[0]]
k0Al = -np.sum((np.log((AlLin - AlBar)[index3]) - np.log(intAl))*time[index3])/np.sum(time[index3]**2)

# According to the model, k0Si should be equal to k0Al:
k0Exp = (k0Si + k0Al)/2     

print 'Intercept and slope SiBar data: ', intSi, '    ', slopeSi, '    ', k0Si
print 'Intercept and slope AlBar data: ', intAl, '    ', slopeAl, '    ', k0Al
print 'Decay rate used for initial guess:                                    ', k0Exp

# De decay rates geven soms nan terug. Moet nog opgelost worden.

In [ ]:
paras0 = np.array([k0Exp, intSi, intSi/intAl, slopeSi, slopeSi/slopeAl, 0])
print 'Inital parameters: ', paras0

timeFull = np.linspace(np.min(time), np.max(time), time.size)
Si0, Al0 = Model(paras0, timeFull)

plt.figure()
lineSiBar = plt.plot(time, SiBar, 'r,')
lineSi0 = plt.plot(time, Si0, 'r--', label = '$Si$')
lineAlBar = plt.plot(time, AlBar, 'b,')
lineAl0 = plt.plot(time, Al0, 'b--', label = '$Al$')
plt.title('Measurement data & inital guess')
plt.xlabel('Time')
plt.ylabel('Concentration ($\mu$mol/L)')
plt.legend(loc = 4)
plt.show()

# Calculate the three models

We now calcuate an optimal set of parameters for $n = 1, 2$ and $3$ and compare the results.

### n = 1:

In [ ]:
%%time

#
n = 1
paras0 = np.concatenate((k0Exp*np.ones(n), intSi/n*np.ones(n), intSi/intAl*np.ones(n), [slopeSi], [slopeSi/slopeAl], [0]))

# Optimization:
runs = 3    # Number of runs
for i in range(runs):
    results = StochasticNewtonMCMC(paras0, SiBar, AlBar, time, samples = 350)    # Number of iterations
    results['x'] = Sort_By_k(results['x'])
    print results['x'], results['fun'], results['flag_hess'], results['flag_naninf']
    if i == 0:
        results1 = results
    elif results['fun'] < results1['fun']:
        results1 = results
Si1, Al1 = Model(results1['x'], time)
Si1Full, Al1Full = Model(results1['x'], timeFull)

# Make a plot of the model:
plt.figure()
plt.plot(time, SiBar, 'r,')
plt.plot(timeFull, Si1Full, 'r', label = 'Si')
plt.plot(time, AlBar, 'b')
plt.plot(timeFull, Al1Full, 'b', label = 'Al')
plt.title('Measurements & model (n = 1)')
plt.xlabel('Time')
plt.ylabel('Concentration ($\mu$mol/L)')
plt.legend(loc = 4)

#
k1 = results1['x'][0:n]
AlkExSi1 = results1['x'][n:2*n]
SiAl1 = results1['x'][2*n:3*n]
b1 = results1['x'][3*n]
SiAl01 = results1['x'][3*n + 1]
t01 = results1['x'][3*n + 2]

print '\nThe following parameters were found:'
print '\tk:      \t', k1
print '\tAlkExSi:\t', AlkExSi1
print '\tSi/Al:  \t', SiAl1
print '\tb:      \t', b1
print '\tSi/Al0: \t', SiAl01
print '\tt0:     \t', t01
print ''
print 'Cost: ', results1['fun']
print ''
print results1['flag_hess'], '    ', results1['flag_naninf']

### n = 2:

In [ ]:
%%time

#
n = 2
paras0 = np.array([0., k1[0], 0., AlkExSi1[0], 1., SiAl1[0], b1, SiAl01, t01])

# Optimization:
runs = 3    # Number of runs
for i in range(runs):
    results = StochasticNewtonMCMC(paras0, SiBar, AlBar, time, samples = 350)    # Number of iterations
    results['x'] = Sort_By_k(results['x'])
    print results['x'], results['fun'], results['flag_hess'], results['flag_naninf']
    if i == 0:
        results2 = results
    elif results['fun'] < results2['fun']:
        results2 = results
Si2, Al2 = Model(results2['x'], time)
Si2Full, Al2Full = Model(results2['x'], timeFull)

# Make a plot of the model:
plt.figure()
plt.plot(time, SiBar, 'r,')
plt.plot(timeFull, Si2Full, 'r', label = 'Si')
plt.plot(time, AlBar, 'b')
plt.plot(timeFull, Al2Full, 'b', label = 'Al')
plt.title('Measurements & model (n = 2)')
plt.xlabel('Time')
plt.ylabel('Concentration ($\mu$mol/L)')
plt.legend(loc = 4)

#
k2 = results2['x'][0:n]
AlkExSi2 = results2['x'][n:2*n]
SiAl2 = results2['x'][2*n:3*n]
b2 = results2['x'][3*n]
SiAl02 = results2['x'][3*n + 1]
t02 = results2['x'][3*n + 2]

print '\nThe following parameters were found:'
print '\tk:      \t', k2
print '\tAlkExSi:\t', AlkExSi2
print '\tSi/Al:  \t', SiAl2
print '\tb:      \t', b2
print '\tSi/Al0: \t', SiAl02
print '\tt0:     \t', t02
print ''
print 'Cost: ', results2['fun']
print ''
print results2['flag_hess'], '    ', results2['flag_naninf']

### n = 3:

In [ ]:
%%time

#
n = 3    #Number of runs
paras0 = np.array([0., k2[0], k2[1], 0., AlkExSi2[0], AlkExSi2[1], 1., SiAl2[0], SiAl2[1], b2, SiAl02, t02])

# Optimization:
runs = 3    # Number of runs
for i in range(runs):
    results = StochasticNewtonMCMC(paras0, SiBar, AlBar, time, samples = 350)    # Number of iterations
    results['x'] = Sort_By_k(results['x'])
    print results['x'], results['fun'], results['flag_hess'], results['flag_naninf']
    if i == 0:
        results3 = results
    elif results['fun'] < results3['fun']:
        results3 = results
Si3, Al3 = Model(results3['x'], time)
Si3Full, Al3Full = Model(results3['x'], timeFull)

# Make a plot of the model:
plt.figure()
plt.plot(time, SiBar, 'r,')
plt.plot(timeFull, Si3Full, 'r', label = 'Si')
plt.plot(time, AlBar, 'b,')
plt.plot(timeFull, Al3Full, 'b', label = 'Al')
plt.title('Measurements & model n = 3')
plt.xlabel('Time')
plt.ylabel('Concentration ($\mu$mol/L)')
plt.legend(loc = 4)

#
k3 = results3['x'][0:n]
AlkExSi3 = results3['x'][n:2*n]
SiAl3 = results3['x'][2*n:3*n]
b3 = results3['x'][3*n]
SiAl03 = results3['x'][3*n + 1]
t03 = results3['x'][3*n + 2]

print '\nThe following parameters were found:'
print '\tk:      \t', k3
print '\tAlkExSi:\t', AlkExSi3
print '\tSi/Al:  \t', SiAl3
print '\tb:      \t', b3
print '\tSi/Al0: \t', SiAl03
print '\tt0:     \t', t03
print ''
print 'Cost: ', results3['fun']
print ''
print results3['flag_hess'], '    ', results3['flag_naninf']

# Model Selection

We will perform an Akaike test and an F test in order to determine which model, i.e. $n = 1, 2$ or $3$, best fits the data.

### All parameters:

In [ ]:
print 'Model 1: Cost =', results1['fun']
print '\t[k1, AlkExSi1, SiAl1]:\t\t[', k1[0], ',', AlkExSi1[0], ',', SiAl1[0], ']'
print '\t[b, SiAl0, t0]:\t\t\t[', b1, ',', SiAl01, ',', t01, ']'

print ''

print 'Model 2: Cost =', results2['fun']
print '\t[k1, AlkExSi1, SiAl1]:\t\t[', k2[0], ', ', AlkExSi2[0], ', ', SiAl2[0], ']'
print '\t[k1, AlkExSi1, SiAl1]:\t\t[', k2[1], ', ', AlkExSi2[1], ', ', SiAl2[1], ']'
print '\t[b, SiAl0, t0]:\t\t\t[', b2, ',', SiAl02, ',', t02, ']'

print ''

print 'Model 3: Cost =', results3['fun']
print '\t[k1, AlkExSi1, SiAl1]:\t\t[', k3[0], ', ', AlkExSi3[0], ', ', SiAl3[0], ']'
print '\t[k1, AlkExSi1, SiAl1]:\t\t[', k3[1], ', ', AlkExSi3[1], ', ', SiAl3[1], ']'
print '\t[k1, AlkExSi1, SiAl1]:\t\t[', k3[2], ', ', AlkExSi3[2], ', ', SiAl3[2], ']'
print '\t[b, SiAl0, t0]:\t\t\t[', b3, ',', SiAl03, ',', t03, ']'

## Akaike test

For a least squares problem, the Akaike information is defined as

$$
    AIC = 2\times(\# parameters) + (\# samples)\times\ln\left(\frac{RSS}{\# samples}\right)
$$

A correction can be made in order to account for the difference between $\# parameters$ and $\# samples$:

$$
    AICc = AIC + 2\times\frac{(\# parameters)\times(\# parameters + 1)}{(\# sampels) - (\# parameters) - 1}
$$

This correction becomes important when $(\# parameters)^2 \ll (\# samples)$.

The model with the smallest AICc is the prefered model.

In [ ]:
RSS = np.array([results1['fun'], results2['fun'], results3['fun']])    # The cost function is precisely the RSS.
AIC = np.zeros(3)
AICc = np.zeros(3)
for i in range(3):
    k = 3*(i + 1) + 1                                                  # The variance has to be counted as a parameter as wel.
    AIC[i] = 2*k + nsamples*np.log(RSS[i]/nsamples)
    AICc[i] = AIC[i] + 2.0*k*(k + 1)/(nsamples- k - 1)                 # Corrected AIC information.

for i in range(3):
    if AICc[i] == AICc.min():
        modelAICc = i + 1
        print 'AICc values: ', AICc, ' ==>  Use model', i + 1
        break
        
print 'SELECTED MODEL: n =', n

if n == 1:
    results = results1
elif n == 2:
    results = results2
else:
    results = results3

Si, Al = Model(results['x'], time)
SiFull, AlFull = Model(results['x'], timeFull)

k = results['x'][0:n]
AlkExSi = results['x'][n:2*n]
SiAl = results['x'][2*n:3*n]
b = results['x'][3*n]
SiAl0 = results['x'][3*n + 1]
t0 = results['x'][3*n + 2]

print '\nThe following parameters were found:'

string1 = '[k, AlkExSi, SiAl]'
for i in range(n):
    string2 = string1[:2] + `i + 1` + string1[3:11] + `i + 1` + string1[11:17] + `i + 1` + string1[17:]
    print '\t',string2, '\t[', k[i], ',', AlkExSi[i], ',', SiAl[i], ']'
print '\tb:      \t\t', b
print '\tSi/Al0: \t\t', SiAl0
print '\tt0:     \t\t', t0


# Analysis:

By selecting a random subset of the data points (indicated by the variable $datasize$), we can create a new set of observations for the same soil sample. If we do the optimization for this subset of the data, we will get a new (and slightly different) set of optimal parameters.

If we do this multiple times (indicated by the variable $runs$), we can calculate the average and standerd deviation over all the parameter sets that were found. These values will allow us to calculate a confidence interval for the parameters.

## Confidence interval

Once again we can compute confidence intervals. This time, they do not represent the quality of the fit, but rather the range in which the parameters are likely to fall.

**Remark 1:** We once again use the Bonferroni correction.

**Remark 2:** The check indicates wether or not the parameters estimated by the Monte Carlo method fall within the confidence interval.

In [ ]:
#
datasize = 50               # Number of points we randomly select from all the available data in order to simulate "new" data.
runs = 100                  # Number of times we will simulate "new".

# Allocate output:
X = np.zeros((runs, 3*n + 3))
FUN = np.zeros(runs)
timeRUNS = np.zeros((runs, datasize))
SiRUNS = np.zeros((runs, datasize))
AlRUNS = np.zeros((runs, datasize))

# Constraints for the optimization routine:
bnds = []
for i in range(0, 2*n):
    bnds.append((0., None))
for i in range(2*n, 3*n):
    bnds.append((1e-16, None))
bnds.append((0, None))
bnds.append((1e-16, None))
bnds.append((None, None))
bnds = tuple(bnds)

#
for run in log_progress(range(runs), every = 1):
    # Select part of the data:
    index = np.random.randint(0, nmeas, datasize)
    
    # Select the measurement times:
    timeRUN = data_np[index, 0]
    
    # Select the Si and Al data:
    SiBarRUN = data_np[index, 1]
    AlBarRUN = data_np[index, 2]
    
    # Take a random percentage shift:
    gammaSi = np.random.uniform(-0.11, 0.11)
    gammaAl = np.random.uniform(-0.11, 0.11)
    
    # Add the percentage shift to the data:
    SiBarRUN = (1 + gammaSi)*SiBarRUN                                                           # Here the noise is added to Si.
    AlBarRUN = (1 + gammaAl)*AlBarRUN                                                           # Here the noise is added to Al.
    
    # Perform the optimization: (local should work fine since we use a good initial estimate and since the dataset is smaller)
    resultsRUN = minimize(Cost, results['x'], args = (SiBarRUN, AlBarRUN, timeRUN), bounds = bnds)
    
    # Store results:
    X[run, :] = resultsRUN['x']
    FUN[run] = resultsRUN['fun']
    timeRUNS[run, :] = timeRUN
    SiRUNS[run, :] = SiBarRUN
    AlRUNS[run, :] = AlBarRUN

In [ ]:
# Sort the data:
results['x'] = Sort_By_k(results['x'])                 #Sort results according to AlkExSi, k or SiAl
for run in range(runs):
    X[run, :] = Sort_By_k(X[run, :])                   #Sort results according to AlkExSi, k or SiAl

#
muX = np.mean(X, 0)
sigmaX =  np.std(X, 0, ddof = 3*n + 3)

#
alpha = 0.05/(3*n + 3)                                   # Bonferroni correction.
q = t.ppf(1 - alpha/2, runs - (3*n + 3))
lb = muX - q*sigmaX
ub = muX + q*sigmaX
check = (lb <= results['x']) * (results['x'] <= ub)

# Make dataframe:
if n == 1:
    indices = ['k1', 'AlkExSi1', 'SiAl1', 'b', 'SiAl0', 't0']
elif n == 2:
    indices = ['k1', 'k2', 'AlkExSi1', 'AlkExSi2', 'SiAl1', 'SiAl2', 'b', 'SiAl0', 't0']
else:
    indices = ['k1', 'k2', 'k3', 'AlkExSi1', 'AlkExSi2', 'AlkExSi3', 'SiAl1', 'SiAl2', 'SiAl3', 'b', 'SiAl0', 't0']      
d = {'Lowerbound':lb, 'Mean':muX, 'Upperbound':ub, 'Width':np.abs(ub - lb), 'std':sigmaX, 'Monte Carlo value':results['x'],
     'Monte Carlo between the bounds':check}
cols = ['Lowerbound', 'Mean', 'Upperbound', 'Width', 'std', 'Monte Carlo value', 'Monte Carlo between the bounds']
df2 = pd.DataFrame(d, columns = cols, index = indices)

df2

## Percentage bands:

To each subset of the data we added a random percentage shift. We can plot the percentage bands around the "optimal" curve found with the Monte Carlo analysis and see if the subsets that we used fall within the maximum percentage bands.

In [ ]:
# The maximum size of the percentage bands:
# (can be different for Si and Al, should be the same value used to add noise to the random subsets)
gammaSi = 0.11    #Noise chosen
gammaAl = 0.11    #Noise chosen

# We will plot every 10th subset of the data that was taken:
index = 10 

# Plot for Si:
plt.figure()
plt.plot(timeFull, (1 + gammaSi)*SiFull, 'r--', linewidth = 2)
plt.plot(timeFull, SiFull, 'r', label = 'Si', linewidth = 2)
plt.plot(timeFull, (1 - gammaSi)*SiFull, 'r--', linewidth = 2)
plt.plot(time, SiBar, 'r,')
for i in range(0, runs, index):
    plt.plot(timeRUNS[i, :], SiRUNS[i, :], 'd', label = 'Run ' + str(i + 1))

# Add legend: (if it's too big, you can comment it out)
plt.legend(loc = 4, bbox_to_anchor = (1.15, -0.1), ncol = 4)

# Plot for Al:
plt.figure()
plt.plot(timeFull, (1 + gammaAl)*AlFull, 'r--', linewidth = 2)
plt.plot(timeFull, AlFull, 'r', label = 'Al', linewidth = 2)
plt.plot(timeFull, (1 - gammaAl)*AlFull, 'r--', linewidth = 2)
plt.plot(time, AlBar, 'r,')
for i in range(0, runs, index):
    plt.plot(timeRUNS[i, :], AlRUNS[i, :], 'd', label = 'Run ' + str(i + 1))

# Add legend: (if it's too big, you can comment it out)
plt.legend(loc = 4, bbox_to_anchor = (1.15, -0.1), ncol = 4)